# Task 2 - User Engagement Analysis

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [3]:
df = pd.read_excel('cleaned_data.xlsx')
df

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,13114483460844900352,2019-04-04 12:01:18,770.0000,2019-04-25 14:35:31,662.00000,1.823652e+06,2.082014e+14,3.366496e+10,3.552121e+13,9164566995485190,...,1.585461e+07,2.501332e+06,8.198936e+06,9.656251e+06,2.780823e+08,1.434415e+07,1.717444e+08,8.814393e+06,3.674974e+07,3.088796e+08
1,13114483482878900224,2019-04-09 13:04:04,235.0000,2019-04-25 08:15:48,606.00000,1.365104e+06,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,2.024740e+07,1.911173e+07,1.833841e+07,1.722713e+07,6.087501e+08,1.170709e+06,5.269042e+08,1.505514e+07,5.380039e+07,6.533850e+08
2,13114483484080500736,2019-04-09 17:42:11,1.0000,2019-04-25 11:58:13,652.00000,1.361762e+06,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,1.972566e+07,1.469958e+07,1.758779e+07,6.163408e+06,2.295846e+08,3.956300e+05,4.106926e+08,4.215763e+06,2.788364e+07,2.798073e+08
3,13114483485442799616,2019-04-10 00:31:25,486.0000,2019-04-25 07:36:35,171.00000,1.321509e+06,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,2.138812e+07,1.514664e+07,1.399465e+07,1.097942e+06,7.995382e+08,1.084972e+07,7.490399e+08,1.279728e+07,4.332422e+07,8.460285e+08
4,13114483499480700928,2019-04-12 20:10:23,565.0000,2019-04-25 10:40:32,954.00000,1.089009e+06,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,1.525938e+07,1.896287e+07,1.712458e+07,4.152180e+05,5.277072e+08,3.529801e+06,5.507095e+08,1.391032e+07,3.854281e+07,5.691386e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7277825670196679680,2019-04-29 07:28:42,451.0000,2019-04-30 06:02:33,214.00000,8.123000e+04,2.082022e+14,3.365069e+10,3.548311e+13,D20434A,...,1.619167e+07,1.176343e+07,1.788370e+07,1.967816e+07,5.266097e+08,9.197207e+06,3.264510e+06,1.348742e+07,5.762885e+07,5.741753e+08
149997,7349883264234609664,2019-04-29 07:28:42,483.0000,2019-04-30 10:41:33,187.00000,9.797000e+04,2.082019e+14,3.366345e+10,3.566051e+13,D10223C,...,1.387723e+07,8.288284e+06,1.935015e+07,2.129315e+07,6.268931e+08,4.735033e+06,7.121804e+08,2.457758e+06,3.913508e+07,6.666488e+08
149998,13114483573367300096,2019-04-29 07:28:43,283.0000,2019-04-30 10:46:12,810.00000,9.824900e+04,2.082017e+14,3.362189e+10,3.572121e+13,T51102A,...,2.266051e+07,1.855903e+06,9.963942e+06,5.065760e+06,5.535395e+08,1.339432e+07,1.211009e+08,1.131473e+07,3.491222e+07,5.927864e+08
149999,13114483573367300096,2019-04-29 07:28:43,696.0000,2019-04-30 10:40:34,327.00000,9.791000e+04,2.082021e+14,3.361962e+10,8.618620e+13,L88342B,...,8.817106e+06,8.305402e+06,3.322253e+06,1.317259e+07,3.525370e+08,2.529475e+06,8.147131e+08,1.406930e+06,2.962610e+07,3.718959e+08


In [12]:
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])

#check for missing values in MSISDN/Number before grouping

if df['MSISDN/Number'].isnull().any():
    print("Waring: Missing Values in 'MSISDN/Number' column.")

#caluclate session frequency for each user
session_frequency = df.groupby('MSISDN/Number')['Bearer Id'].count().reset_index()
session_frequency.rename(columns={'Bearer Id': 'Session Frequency'}, inplace = True)

#Merge the Session Frequency back into the original DataFrame
df = pd.merge(df, session_frequency, on='MSISDN/Number', how='left')

#Display the result Dataframe with the 'Seission Frequency', column
print(df[['MSISDN/Number', 'Session Frequency']].head())



   MSISDN/Number  Session Frequency
0   3.366496e+10                  2
1   3.368185e+10                  2
2   3.376063e+10                  1
3   3.375034e+10                  1
4   3.369980e+10                  1


In [13]:
# Calculate session duration in seconds
df['Session Duration (s)'] = df['Dur. (ms)'] / 1000

# Display the result
print(df[['MSISDN/Number', 'Session Duration (s)']].head())

   MSISDN/Number  Session Duration (s)
0   3.366496e+10              1823.652
1   3.368185e+10              1365.104
2   3.376063e+10              1361.762
3   3.375034e+10              1321.509
4   3.369980e+10              1089.009


In [16]:
#Calculate Session Total Traffic in Bytes
df['Session Total Traffic (Bytes)'] = df['Total DL (Bytes)'] + df['Total UL (Bytes)']

print(df[['MSISDN/Number', 'Session Total Traffic (Bytes)']].head())

   MSISDN/Number  Session Total Traffic (Bytes)
0   3.366496e+10                    345629377.0
1   3.368185e+10                    707185356.0
2   3.376063e+10                    307690973.0
3   3.375034e+10                    889352748.0
4   3.369980e+10                    607681403.0


In [44]:
engagement_Materics = df.groupby('MSISDN/Number').agg({'Bearer Id': 'count', 'Dur. (ms)': 'sum', 'Session Total Traffic (Bytes)': 'sum'})
engagement_Materics = engagement_Materics.rename(columns={'Bearer Id': 'Session Frequency', 'Dur. (ms)': 'Session Duration (ms)'})
engagement_Materics.sort_values(by=['Session Frequency'], ascending=False).head(10)

,Session Frequency,Session Duration (ms),Session Total Traffic (Bytes)
MSISDN/Number,,,
4.188282e+10,1066,7.255100e+07,5.317447e+11
3.362632e+10,18,8.791927e+06,7.971167e+09
3.362578e+10,17,1.855375e+07,8.499621e+09
3.361489e+10,17,9.966898e+06,8.846226e+09
3.365973e+10,16,4.035428e+06,7.705863e+09
3.367588e+10,15,4.865947e+06,7.891111e+09
3.376054e+10,15,9.279434e+06,8.514774e+09
3.366716e+10,13,8.744914e+06,5.618394e+09
3.360452e+10,12,5.207990e+06,5.487855e+09


# Normalize each engagement metric and run a k-means (k=3) to classify customers into three groups of engagement.

In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

scaler=StandardScaler()
Normalized_engagement_metrics=scaler.fit_transform(engagement_Materics[['Session Frequency', 'Session Duration (ms)', 'Session Total Traffic (Bytes)']])

kmeans=KMeans(n_clusters=3, random_state=0).fit(Normalized_engagement_metrics)
engagement_Materics['Cluster'] = kmeans.labels_
engagement_Materics.head().reset_index()

AttributeError: 'NoneType' object has no attribute 'split'